## 1. Get the data we are going to use
In this case, the data comes from the John Hopkins CSSE GitHub repo: https://github.com/CSSEGISandData/COVID-19

In [1]:
#cd("/Users/michael/work/data/COVID-19")
cd("/Users/michael/work/GitHub/COVID-19/csse_covid_19_data/csse_covid_19_time_series")

In [2]:
using CSV
using Plots

In [3]:
function getdata(country::String="Australia", state::String="Western Australia", file::String="time_series_covid19_confirmed_global.csv")

x=[]
for row in CSV.File(file)
    if row[2]==country
        if length(state)==0 && row[1]===missing
            x=row
        elseif row[1]==state
            x=row
        end
    end
end
    
z=Array{Int64,1}(undef,length(x)-4)
    
for (index, value) in enumerate(x)
    if index>4
        z[index-4]=value
    end    
end

z
end
    

getdata (generic function with 4 methods)

In [4]:
file="time_series_covid19_confirmed_global.csv"
allstates=["Australian Capital Territory" "New South Wales"  "Northern Territory" "Queensland" "South Australia" "Tasmania" "Victoria" "Western Australia"]


1×8 Array{String,2}:
 "Australian Capital Territory"  …  "Victoria"  "Western Australia"

In [5]:
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/michael/.julia/packages/Plots/cc8wh/src/backends.jl:363


Plots.PlotlyBackend()

In [6]:
z=[]
for state in allstates
    z=push!(z,getdata("Australia",state))
end

In [7]:
ddays=CSV.File(file)[1]
ddays=propertynames(ddays,4)[5:end]
ddays=String.(ddays)
ndays=length(ddays)

87

Plot data for various Australian states

In [8]:
plot(ddays,z,label=allstates,lw=3,size=(1000,600))

<!DOCTYPE html>
 
 
 Plots.jl

In [16]:
othercountries=["France" "Germany" "Spain" "Greece" "Italy" "United Kingdom" "Korea, South" "New Zealand"]

1×8 Array{String,2}:
 "France"  "Germany"  "Spain"  "Greece"  …  "Korea, South"  "New Zealand"

In [17]:
x=[]
for places in othercountries
    x=push!(x,getdata(places,""))
end

Or, we could look at other countries (note that there is a subtelity here that becomes somewhat fiddly when data for dependencies or states/provinces/regions are given seperately. Also, the US is not here, they're in a seperate file. 

In [18]:
plot(ddays,x,label=othercountries,lw=3,size=(1000,600))

<!DOCTYPE html>
 
 
 Plots.jl

For now, we'll return to Australia, and in particular WA

In [12]:
y=z[8];

Our criteria for reaching the transition between growth and decay is the infelxion point of the curve, that is, when the second derivative changes sign.

In [13]:
ddt(z,zt)=count(z->z>0, z[1:zt])+ count(z->z<0, z[zt+1:end])
~,tpday=findmax([ddt(diff(diff(y)),nx) for nx in 1:(ndays-2)])

(26, 59)

In [14]:
#this is the turning point between exponential growth and decay. totItp total infections at day tpday
totItp=y[tpday+1]

90

In [15]:
plot(1:tpday+1,y[1:tpday+1],lw=4,label="growth phase")
plot!(tpday+1:ndays,y[tpday+1:ndays],lw=4,label="plateau")

<!DOCTYPE html>
 
 
 Plots.jl

In [57]:
#'reasonable' parameters
gridsize=1450
pop=gridsize^2
p0=0.2 #a guess - tuned to match observed data 
p2=1/12 #revised infection rate with distancing measures
q=1/7 #"up to" two weeks
r0=1/14 #about two weeks for mild, 3-6 for severe
r2=1/4 #revised removal rate (now due to testing and isolation)
 #   ndays=120 #120 prediction (from patient(s) zere
 #   nsims=50 #50 runs
nseeds=5 #probably too many, consider dropping.



#got that working ... put it all together
function episim(net1, net2, feth::Int64=100, predays::Int64=40, postdays::Int64=80, nsims::Int64=50)
    #feth is the threhold to switch between growth and control
    
    state=Array{Int8,2}(undef,1,pop) #this is a bit wasteful, there must be a better categorical way to do this...
    state[1:pop].=1;
    state[rand((1:pop),(1,nseeds))].=2; #seeds should be exposed cases
    
    ndays=predays+postdays
    st=Array{UInt64,2}(undef,ndays,nsims)
    ex=Array{UInt64,2}(undef,ndays,nsims)
    fe=Array{UInt64,2}(undef,ndays,nsims)
    rm=Array{UInt64,2}(undef,ndays,nsims)
    st[1,1:nsims] .= count(x->x==1,state)
    ex[1,1:nsims] .= count(x->x==2,state)
    fe[1,1:nsims] .= count(x->x==3,state)
    rm[1,1:nsims] .= count(x->x==4,state)
    
    iter = ProgressBar(1:nsims)
    for j in iter
        net=net1
        #reinitialise the state vector
        state=Array{Int8,2}(undef,1,pop) #this is a bit wasteful, there must be a better categorical way to do this...
        state[1:pop].=1;
        state[rand((1:pop),(1,nseeds))].=2; #seeds should be exposed cases
        r=r0
        p=p0
        
        i=1
        notdoneyet=true
        while i<=(predays+postdays)

            for v in vertices(net)
                if state[v]==1
                    for n in all_neighbors(net, v)
                        if state[n]==3 && rand(Float64).<p
                            state[v]=2   #susceptible becomes exposed
                        elseif state[n]==14 && rand(Float64).<p
                            state[v]=2   #susceptible becomes exposed
                        end
                    end
                elseif state[v]==2 && rand(Float64).<q
                    state[v]=13 #exposed becomes infectious
                elseif state[v]==3 && rand(Float64).<r
                    state[v]=14 #infectious becomes removed
                end
            end
            state[state.>10]= state[state.>10] .- 10
            st[i,j] = count(x->x==1,state)
            ex[i,j] = count(x->x==2,state)
            fe[i,j] = count(x->x==3,state)
            rm[i,j] = count(x->x==4,state)
            
            #########################################
            #switch network after 300 infections
            inftot = pop-(st[i,j]+ex[i,j])
            if notdoneyet && inftot>feth 
                net=net2 #change network structure
                r=r2     #change the removal rate
                p=p2     #and the infection rate
                #realign everything - this is clunky, should be a nicer way to achieve the same thing...
                if i>predays
                        shft=i-predays
                        for k in 1:predays
                            st[k,j]=st[k+shft,j]
                            ex[k,j]=ex[k+shft,j]
                            fe[k,j]=fe[k+shft,j]
                            rm[k,j]=rm[k+shft,j]
                        end
                elseif i<predays
                        shft=predays-i
                        for k in (predays-shft):-1:1
                            st[k+shft,j]=st[k,j]
                            ex[k+shft,j]=ex[k,j]
                            fe[k+shft,j]=fe[k,j]
                            rm[k+shft,j]=rm[k,j]
                        end
                        for k in 1:shft
                            st[k,j]=pop
                            ex[k,j]=0
                            fe[k,j]=0
                            rm[k,j]=0
                        end
                end 
                i=predays
                notdoneyet=false
            end
            #########################################
            i += 1
        end
    end
    
    return st, ex, fe, rm

end

episim (generic function with 5 methods)

In [24]:
using LightGraphs

{2102500, 4205000} undirected simple Int64 graph

In [56]:
bamodel=barabasi_albert(gridsize^2, 3, 2)
lattice=LightGraphs.grid((gridsize,gridsize),periodic=true)
wattstrog=watts_strogatz(gridsize^2, 4, 0.05) 

{2102500, 4205000} undirected simple Int64 graph

In [27]:
using ProgressBars

In [ ]:
St,Et,It,Rt=episim(bamodel,wattstrog,totItp, tpday, 90, 100)

3.0%┣█▎                                          ┫ 3/100 [00:22<18:05, 0.1 it/s]

In [55]:
plot(1:tpday+1,y[1:tpday+1],lw=4,label="growth phase")
plot!(tpday+1:ndays,y[tpday+1:ndays],lw=4,label="plateau")
plot!(pop .- (St+Et),ylims=(0,10000))

<!DOCTYPE html>
 
 
 Plots.jl

In [51]:
function plotquantiles(y,col,labl,qnt::Float64=0.3)
    nt,ny=size(y)
    low=Array{Float64,1}(undef,nt)
    mid=Array{Float64,1}(undef,nt)
    hig=Array{Float64,1}(undef,nt)
    for i in 1:nt
        low[i],mid[i],hig[i] = quantile(y[i,:],[0.5-qnt, 0.5, 0.5+qnt])
    end
    plot!(1:nt,mid,grid=false,ribbon=(mid-low,hig-mid),fillalpha=.25,lw=3, seriescolor=col, label=labl)
end

plotquantiles (generic function with 2 methods)

In [53]:
plot()
plotquantiles(pop .- (Et+St),:red,"symptomatic")




<!DOCTYPE html>
 
 
 Plots.jl